# Scopes and Environments: A Deeper Look

The purpose of this chapter is to take a deeper look at how scopes and
environment work in the Lettuce language with arithmetic expressions, boolean expressions and let bindings. The lecture will be structured as follows:
  - We will first recall notions of scope and shadowing.
  - Next, we will look at the environment implemented using Map data structures in the previous chapter introducing the Lettuce language with let bindings.
  - In particular, we examine how the recursive interpreter produces the correct handling of scopes. 
  - Next, we look at alternative implementations that are more realistic choices for interpreters based on linked-lists (mimicing stack) data structure: we will call them "scope chains".
  - Finally, we will examine how scope chains work with the interpreter.

## Language and Abstract Syntax

Let us recall the language we will work with, which will feature a __minimal__ subset of Lettuce language with just the relevant features to illustrate the ideas here.

$$\begin{array}{rcll}
\mathbf{Program} & \rightarrow & TopLevel(\mathbf{Expr}) \\[5pt]
\mathbf{Expr} & \rightarrow & Const(\mathbf{Number}) \\
 & | & Ident(\mathbf{Identifier}) \\
 & | & Plus(\mathbf{Expr}, \mathbf{Expr}) \\
 & | & Geq (\mathbf{Expr}, \mathbf{Expr}) \\
 & | & IfThenElse(\mathbf{Expr}, \mathbf{Expr}, \mathbf{Expr}) & \text{if (expr) then expr else expr} \\
 & | & Let( \mathbf{Identifier}, \mathbf{Expr}, \mathbf{Expr}) & \text{let identifier = expr in expr} \\
 \end{array}$$
 
 The chosen fragment has constants, identifiers, a "token" arithmetic expression, a "token" comparison operator, if then else and let bindings. However, the ideas presented can easily extend to the larger set of expressions in the language that include other arithmetic operators such as multiplication, division, sine, cosine and so on; and boolean operators such as equality comparison, and, or, and not.
 
 ### Definition in Scala
 
 We translate these definitions in Scala in the "standard" manner.

In [1]:
sealed trait Program
sealed trait Expr
case class TopLevel(e: Expr) extends Program
case class Const(f: Double) extends Expr
case class Ident(s: String) extends Expr 
case class Plus(e1: Expr, e2: Expr) extends Expr
case class Geq(e1: Expr, e2: Expr) extends Expr
case class IfThenElse(cond: Expr, thenBranch: Expr, elseBranch: Expr) extends Expr
case class Let(s: String, e1: Expr, e2: Expr) extends Expr

defined trait Program
defined trait Expr
defined class TopLevel
defined class Const
defined class Ident
defined class Plus
defined class Geq
defined class IfThenElse
defined class Let

## Scopes and Shadowing

We started looking at the notion of scopes in the previous chapter on Lettuce. Let us recall the concepts briefly.

Programs define and use identifiers (these can be constant vals or mutable vars) all over the place. The scope of a definition/declaration in a program specifies __where__ in the program that particular definition/declaration is used. 

We have already learned that for a let binding of the form

~~~
let identifier = expression1 in 
    expression2
~~~

The scope of `identifier` is restricted to `expression2`. I.e, `identifier` starts being "in scope" when we start evaluating `expression2` and goes "out of scope" when the evaluation of `expression2` concludes.


### Example 1

~~~
let x = 10 in (* declaration/definition of x *)
  let y = x + 10 in (* declaration of y *)
     x + y - 20
~~~

Notice that in the example above, there are two declarations/definitions using let bindings, as noted in the comments. There are two usages of `x` in the program and one usage of `y`. The following two points should be clear to the reader:
  - Which declaration/definition is being referred to by the respective usages of `x` and `y` in the code.
  - Also, it is clear that every usage of `x` or `y` refers to a declaration that is "in scope" a the "time of usage".


### Example 2

~~~
let x = 10 in 
   let y = ( let z = 20 in 
              x + z ) 
      in 
         x - y
~~~

There are three identifier declarations in this example.
  - `let x = 10 in ..`
  - `let y = (...) ` 
  - `let z = 20 in ..`
  
The reader should convince themselves that 
  - Each usage of `x`, `y` and `z` in the code refers to a declaration that is "in scope" at the "time of usage".
  - It is clear which declaration a particular usage refers to.
  

### Example 3

Let us modify example 2 slightly.


~~~
let x = 10 in 
   let y = ( let z = 20 in 
              x + z ) 
      in 
         x - y + z (* Usages of x, y and z *)
~~~

Is there an issue in this program? 

  - Yes, the usage of `z` at the very last line of the program is problematic. We know that `z` is not in scope for this particular usage.


## Shadowing

We will now examine the issue of "shadowing" wherein a declaration can override/shadow a previous declaration of an identifier with the same name that is currently in scope.

### Shadowing Example 1

~~~
let x = 20 in (* Declaration 1 *) 
  let x = 40 in  (* Declaration 2*)
    x + 30 (* Usage of x *)
~~~
 
Note that `x` is declared twice in two successive let bindings to 20 and 40, respectively. However, according to the semantics for let binding, we note that this program evaluates to `70`.

Declaration 2 is said to __shadow__ declaration 1, and we allow this kind of shadowing in Lettuce. Here is the same code translated to scala.
 

In [2]:
{ 
  val x = 20;
     {
         val x = 40; // This declaration shadows the previous one.
         println("x + 30 = ", x+30) // Notice x is 40 here.
     }
}

(x + 30 = ,70)


x: Int = 20

### Shadowing Example 2

Consider the example below.

~~~
let x = 20 in (* Declaration 1 *) 
  let y = (
         let x = 45 in  (* Declaration 2 of x *)
            x + 20 (* Usage 1 of x *)
          )  in  (* Declaration 2 goes out of scope here *)
      x + y (* Usage 2 of x *)
~~~

Notice that declaration 2 of `x` shadows the previous declaration 1. 
Also, usage 2 of `x` pertains to declaration 1 since declaration 2 goes out of scope as indicated in the program. 

This leads to a key insight:

  - Once the shadowing declaration is out of scope, we have to revert back to the previous declaration.

This happens in scala as well.

In [3]:
{
    val x = 20;
    val y = {
        val x = 45; // This declaration shadows the previous one
        x + 20
    } // The second declaration of x goes out of scope here.
    println("x=", x) // Note here that the previously shadowed declaration comes back in scope
    println("x+ y = ", x + y) // Should print 85
}

(x=,20)
(x+ y = ,85)


x: Int = 20
y: Int = 65

## Implementing Shadowing Correctly

Let us now examine the previous interpreter for Lettuce from last lecture. The key choice made there was to implement environment as a `Map[String, Value]` from names of identifiers to their values.

Here is a simplified implementation for our subset of the language.

In [4]:
/* Values in our language can be numbers, booleans and error */
sealed trait Value
case class BoolValue(b: Boolean) extends Value
case class NumValue (f: Double) extends Value
case object Error extends Value

/* An environment is a map from variable names to values */

type Environment = Map[String, Value] // Map will not contain Error

defined trait Value
defined class BoolValue
defined class NumValue
defined object Error
defined type Environment

In [5]:
def evalExpr( e: Expr, env: Environment): Value =  e match {
    case Const(f: Double) => { NumValue(f) }
    case Ident(s: String) =>  { 
             if (env.contains(s)){ env(s) }
             else {
                 println(s"Fatal error in evalExpr: Identifier $s is not known in current scope")
                 Error
             }
    }
    case Plus(e1, e2) => {
        val v1 = evalExpr(e1, env)
        v1 match {
            case NumValue(f1) => { /* e1 evaluates to a number */
                val v2 = evalExpr(e2, env)
                v2 match {
                    // Both e1 and e2 evaluate to numbers
                    case NumValue(f2) => NumValue(f1 + f2) // Plus happens here.
                    case _ => Error // v2 is not a numerical value, cannot add
                }
            }
            case _ => Error // v1 is not a numerical value, cannot add
        }
    }
    
    case Geq(e1, e2) => {
        val v1 = evalExpr(e1, env)
        v1 match {
            case NumValue(f1) => {
                val v2 = evalExpr(e2, env)
                v2 match {
                    case NumValue(f2) => BoolValue(f1 >= f2)
                    case _ => Error
                }
            }
            case _ => Error
        }
    }
    
    case IfThenElse(condExpr, thenBranch, elseBranch) => {
        val vCond = evalExpr(condExpr, env)
        vCond match {
            case BoolValue(true) => evalExpr(thenBranch, env)
            case BoolValue(false) => evalExpr(elseBranch, env)
            case _ => Error
        }
    }
    
    case Let(ident, e1, e2) => { // let ident = e1 in e2
        val v1 = evalExpr(e1, env)
        v1 match {
            case Error => Error
            case _ => {
                val newEnv = env + (ident -> v1)
                // env is still unchanged
                // newEnv "copies" env over and binds identifier to v1
                evalExpr(e2, newEnv)
            }
        }
    }
}

defined function evalExpr

Did the above implementation handle scopes and shadowing correctly? Let us try some of those examples and see if the answers we obtain match with our intuition.

In [6]:
/* Example 1 (modified to use + instead of -)
let x = 10 in (* declaration/definition of x *)
  let y = x + 10 in (* declaration of y *)
     x + y + 20
     */

val x = Ident("x")
val y = Ident("y")
val ten = Const(10.0)
val ex1 = Let("x", ten, Let("y", Plus(x, ten), Plus(Plus(x,y), Const(20.0))))

val res = evalExpr(ex1, Map.empty)
print(s"Result of Eval: $res")


Result of Eval: NumValue(50.0)

x: Ident = Ident("x")
y: Ident = Ident("y")
ten: Const = Const(10.0)
ex1: Let = Let(
  "x",
  Const(10.0),
  Let(
    "y",
    Plus(Ident("x"), Const(10.0)),
    Plus(Plus(Ident("x"), Ident("y")), Const(20.0))
  )
)
res: Value = NumValue(50.0)

In [7]:
/* Example 2 (modified with + instead of -)
let x = 10 in 
   let y = ( let z = 20 in 
              x + z ) 
      in 
         x + y
         */

val x = Ident("x")
val y = Ident("y")
val z = Ident("z")
val ten = Const(10.0)
val twenty = Const(20.0)

val innerLetZ = Let("z", twenty, Plus(x, z))
val letY = Let("y", innerLetZ, Plus(x, y))
val example2 = Let("x", ten, letY)

val res2= evalExpr(example2, Map.empty)

println(s"Result = $res2")

Result = NumValue(40.0)


x: Ident = Ident("x")
y: Ident = Ident("y")
z: Ident = Ident("z")
ten: Const = Const(10.0)
twenty: Const = Const(20.0)
innerLetZ: Let = Let("z", Const(20.0), Plus(Ident("x"), Ident("z")))
letY: Let = Let(
  "y",
  Let("z", Const(20.0), Plus(Ident("x"), Ident("z"))),
  Plus(Ident("x"), Ident("y"))
)
example2: Let = Let(
  "x",
  Const(10.0),
  Let(
    "y",
    Let("z", Const(20.0), Plus(Ident("x"), Ident("z"))),
    Plus(Ident("x"), Ident("y"))
  )
)
res2: Value = NumValue(40.0)

In [8]:
/* 
  Example 3: modified with + instead of -
  This will not execute correctly.
  
  let x = 10 in 
   let y = ( let z = 20 in 
              x + z ) (* z usage numero 1*) 
      in 
         x + y + z (* Usages of x, y and z *) */


val x = Ident("x")
val y = Ident("y")
val z = Ident("z")
val ten = Const(10.0)
val twenty = Const(20.0)


val innerLetZ = Let("z", twenty, Plus(x, z))
val letY = Let("y", innerLetZ, Plus(Plus(x, y),z))
val example3 = Let("x", ten, letY)

val res3 = evalExpr(example3, Map.empty)

Fatal error in evalExpr: Identifier z is not known in current scope


x: Ident = Ident("x")
y: Ident = Ident("y")
z: Ident = Ident("z")
ten: Const = Const(10.0)
twenty: Const = Const(20.0)
innerLetZ: Let = Let("z", Const(20.0), Plus(Ident("x"), Ident("z")))
letY: Let = Let(
  "y",
  Let("z", Const(20.0), Plus(Ident("x"), Ident("z"))),
  Plus(Plus(Ident("x"), Ident("y")), Ident("z"))
)
example3: Let = Let(
  "x",
  Const(10.0),
  Let(
    "y",
    Let("z", Const(20.0), Plus(Ident("x"), Ident("z"))),
    Plus(Plus(Ident("x"), Ident("y")), Ident("z"))
  )
)
res3: Value = Error

In [9]:
/* Shadowing example 1
let x = 20 in (* Declaration 1 *) 
  let x = 40 in  (* Declaration 2*)
    x + 30 (* Usage of x *)
*/

val x = Ident("x")
val innerLet = Let("x", Const(40.0), Plus(x, Const(30.0)))
val shadowExample1 = Let("x", Const(20.0), innerLet)

val resEx1 = evalExpr(shadowExample1, Map.empty)
println(s"result = $resEx1")

result = NumValue(70.0)


x: Ident = Ident("x")
innerLet: Let = Let("x", Const(40.0), Plus(Ident("x"), Const(30.0)))
shadowExample1: Let = Let(
  "x",
  Const(20.0),
  Let("x", Const(40.0), Plus(Ident("x"), Const(30.0)))
)
resEx1: Value = NumValue(70.0)

In [10]:
/*  Shadowing example2 (modified)
let x = 20 in (* Declaration 1 *) 
  let y = ( 
         let x = 45 in  (* Declaration 2 of x *)
            x + 20 (* Usage 1 of x *)
          )    (* Declaration 2 goes out of scope here *)
    in 
      x  (* Usage 2 of x *)
      */

val x = Ident("x")
val y = Ident("y")
def c(f:Double)= Const(f)

val innerLet1 = Let("x", c(45.0), Plus(x, c(20.0)))
val letY = Let("y", innerLet1, x)

val shadowEx2 = Let("x", c(20.0), letY)

val resEx2 = evalExpr(shadowEx2, Map.empty)

x: Ident = Ident("x")
y: Ident = Ident("y")
defined function c
innerLet1: Let = Let("x", Const(45.0), Plus(Ident("x"), Const(20.0)))
letY: Let = Let(
  "y",
  Let("x", Const(45.0), Plus(Ident("x"), Const(20.0))),
  Ident("x")
)
shadowEx2: Let = Let(
  "x",
  Const(20.0),
  Let("y", Let("x", Const(45.0), Plus(Ident("x"), Const(20.0))), Ident("x"))
)
resEx2: Value = NumValue(20.0)

We see in each example, the implementation of shadowing is correct. How did we manage this in our implementation?

  - The answer is very subtle actually. We successfully piggybacked on the fact that Scala has a stack for handling recursive function calls. We used that stack implicitly in our implementation. To see why, let us go with the last example in detail.
  

~~~
let x = 20 in  
  let y = ( 
         let x = 45 in  (
            x + 20 
          )  in  
      x  
~~~

Let us give names to the (abstract syntax tree) for various subexpressions here.
  - Call the entire program as the expression `e`.
  - Call the entire subexpression  `let y  = ( let x = 45 in x + 20) in x` as `e1`.
  - Call the subexpression `let x = 45 in x + 20` as `e2`
  - Call the subexpression `x + 20` as `e3`
  - Call the subexpression `x` at the very last line of the program `e4`.
  
![Illustration of the execution](interpreter-eval.png)


Let us now trace what happens when we call  `evalExpr(e, Map.empty)`. 
The figure above illustrates what happens pictorially. The reader will find it very helpful to consult the picture.
  - First, we get a recursive call to `evalExpr(Const(20), Map.empty)` which yields the result `NumValue(20)`.
  - Next, we get a recursive call to `evalExpr(e1, env1)` where `env1` is the enviroment $\{x \mapsto NumValue(20)\}$
    - `e1` is a let binding of the form: `let y = e2 in e4`.
    - In turn, this calls `evalExpr(e2, env1)`.
      - recall `e2` itself is  `let x = 45 in x + 20`.
      - `evalExpr(Const(45), env1)` returns `NumValue(45)`.
      - We create an environment `env2` with $\{ x \mapsto NumValue(45) \}$, obtained by updating `env1` with the new binding that maps `x` to `NumValue(45)`. __This is where shadowing happens__
      - In turn, we get `evalExpr(e3, env2)` which evaluates to `NumValue(65)`.
    - Note that we return back to the call `evalExpr(e1, env1)` with the value `65`. The environment `env2` was created in the stack during this recursive call in scala but it no longer exists once the recursion has returned.
    - The environment is `env3` is created by adding the binding $y \mapsto NumValue(65)$ to `env1`. Thus, `env3` is the environment $\{ x \mapsto NumValue(20), y \mapsto NumValue(65) \}$.
  - Finally, the expression `e4` which is simply `x` is executed under `env3`. 

The example illustrates how we got the notion of scoping and shadowing correct simply by writing a semantic rule and implementing it in Scala using recursion. 

An alternative that helps us control and understand this process better is to implement environments as a stack that is in turn realized by a linked list. This allows us to gain some efficiency since we will share various parts of the environment across recursive calls. 



## Environment as an Abstract Data Type

We will first talk about environments as an "abstract data type" wherein the details of the implementation are not important to us but we care intimately about what operations an environment supports. This will allow us to carefully model the environment.

  - The function `emptyEnvironment()` must create an empty environment for us.
  
  - The function `update(identifier: String, v: Value, oldEnvironment: Environment): Environment` must return a new environment that updates an existing environment by binding the identifier `identifier` to the value `v`. If the identifier is already bound in the `oldEnvironment`, it must now be bound to `v`. 
  
  - The function `lookup(identifier: String, env: Environment): Value` must check if the identifier `identifier` is already bound in the environment `env` and return its value. If it is not bound, it must return `Error`.
  
With this under control, let us see how we have implemented these operations thus far.

In [14]:
/*-- Here is one way to implement an environment as a map -- */


type Environment  = Map[String, Value]

def emptyEnvironment() = Map.empty

def update(identifier: String, v: Value, oldEnv: Environment): Environment = {
    oldEnv + (identifier -> v)
}

def lookup(identifier: String, env: Environment): Value = {
    if (env.contains(identifier)){
        env(identifier)
    } else {
        Error
    }
}


defined type Environment
defined function emptyEnvironment
defined function update
defined function lookup

In [15]:
/*-- Here is another way to implement it as a "linked list" --*/

import scala.annotation.tailrec

type Environment = List[(String,Value)]
def emptyEnvironment():Environment  = Nil

def update(identifier: String, v: Value, oldEnv: Environment): Environment = {
    (identifier, v) :: oldEnv 
}

 @tailrec
final def lookup(identifier: String, env: Environment): Value = env match {
    case Nil => Error
    case (id, v)::rest if (id == identifier) => v
    case _::restEnv => lookup(identifier, restEnv) // Traverse list recursively to find a match
}


import scala.annotation.tailrec


defined type Environment
defined function emptyEnvironment
defined function update
defined function lookup

Note that in the linked list (alternatively stack based) implementation, an environment is implemented as a list of pairs, wherein each element of the list contains a `String` (identifier) and the `Value` that it maps to. This is similar to the map, except that whenever we wish to replace an old binding `x -> 20` with a new binding `x -> 45`, for example, we simply place `("x", 45)` in the list (stack) before the pair `("x", 20)`.
By convention, the first tuple that matches "x" yields its current value. 

For instance, if we had the enviroment : 
~~~
[ ("x", 45), ("y", 65), ("x", 20) ]
~~~
We know that it binds `x` to `45` and `y` to `65`. The advantage of this representation is that it is simple and easier to implement in a low level language where we may not have access to immutable maps as we do in scala.

To complete the discussion, we will implement `evalExpr` using the interface we have defined for environments, above.

In [16]:
def evalExpr( e: Expr, env: Environment): Value =  e match {
    case Const(f: Double) => { NumValue(f) }
    case Ident(s: String) =>  { 
             lookup(s, env) // Call the lookup function
    }
    case Plus(e1, e2) => {
        val v1 = evalExpr(e1, env)
        /* Implement the short circuiting on error properly */
        v1 match {
            case NumValue(f1) => { /* e1 evaluates to a number */
                val v2 = evalExpr(e2, env)
                v2 match {
                    // Both e1 and e2 evaluate to numbers
                    case NumValue(f2) => NumValue(f1 + f2) // Plus happens here.
                    case _ => Error
                }
            }
            case _ => Error
        }
    }
    
    case Geq(e1, e2) => {
        val v1 = evalExpr(e1, env)
        v1 match {
            case NumValue(f1) => {
                val v2 = evalExpr(e2, env)
                v2 match {
                    case NumValue(f2) => BoolValue(f1 >= f2)
                    case _ => Error
                }
            }
            case _ => Error
        }
    }
    
    case IfThenElse(condExpr, thenBranch, elseBranch) => {
        val vCond = evalExpr(condExpr, env)
        vCond match {
            case BoolValue(true) => evalExpr(thenBranch, env)
            case BoolValue(false) => evalExpr(elseBranch, env)
            case _ => Error
        }
    }
    
    case Let(ident, e1, e2) => { // let ident = e1 in e2
        val v1 = evalExpr(e1, env)
        v1 match {
            case Error => Error
            case _ => {
                val newEnv = update(ident, v1, env) // call update function to update environment
                // env is still unchanged
                evalExpr(e2, newEnv)
            }
        }
    }
}

defined function evalExpr

### The Lettuce Playground

For your convenience, the execution of the interpreter may be visualized using a scala implementation called _Lettuce Playground_. It is inspired by a similar implementation in Javascript done by a CSCI student Mr. Jacob Bloom.

https://github.com/sriram0339/LettucePlaygroundScala

The Lettuce playground tool allows you to visualize the execution of Lettuce programs along with the environments.